<h1>Calculate similarity scores</h2>
Before this notebook can be used with the generated answers from medvet we have to add the answers from ChatGPT-4. This answers were generated manually in the ChatGPT interface. 
Used Model: ChatGPT 4 
Used prompt template: "You are an intelligent assistant designed to support veterinarians by providing detailed and specific responses related to veterinary medicine, including diagnosis and treatment. You analyze the provided case and tailor your answers to the specific species and context of the inquiry.  Follow the instructions carefully and explain your answers in detail. ...."

---------------------------
Used model in MedVet: LlaMA 2 7b-chat with 8bit quantization as .gguf format
used prompt template: "Casesolver"

In [1]:
import pandas as pd

# Read the CSV file
answer_data = pd.read_csv("Results/Evaluation MedVet Cases.csv")

# View the first 5 rows
answer_data.head()

,Number,Case,Image,Solution,Source,Answer MedVet,Answer ChatGPT
0,1,Jake is a 3-year-old neutered German shepherd....,-,Periodontal disease is present to some degree ...,https://angelscampvetgroup.com/pet-dental-heal...,"Ah, I see. Based on the information provided,...","\nBased on the symptoms described for Jake, it..."
1,2,Casey is a 14-year-old spayed domestic shortha...,-,If bacteria aren‚Äôt removed from under a pet‚...,https://angelscampvetgroup.com/pet-dental-heal...,Thank you for providing the details of Casey'...,"Based on the symptoms described for Casey, the..."
2,3,Karl is a 5-year-old neutered greyhound. Karl‚...,-,As periodontal disease progresses to stage thr...,https://angelscampvetgroup.com/pet-dental-heal...,Thank you for providing me with the details of...,The symptoms Karl is experiencing suggest a de...
3,4,Dolly was a 14-year-old spayed female Chihuahu...,-,"When periodontal disease becomes advanced, sev...",https://angelscampvetgroup.com/pet-dental-heal...,Thank you for sharing the details of Dolly's ...,\nDolly's clinical presentation and findings f...
4,5,"A 13-year-old, female Shiba dog presented with...",Image Case 5,"In this case, MCT involved subcutaneous bleedi...",https://bvajournals.onlinelibrary.wiley.com/do...,Based on the provided case description and ac...,The information provided in the case of the 13...


<h2>Calculate cosine similarity</h2>

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the CSV file
df = pd.read_csv('Results/Evaluation MedVet Cases.csv')  

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Combine the text from both columns for vectorization
all_text = pd.concat([df['Solution'], df['Answer MedVet'],df['Answer ChatGPT']], axis=0)

# Fit the vectorizer on all the text
vectorizer.fit(all_text)

# Transform text1 and text2 into TF-IDF vectors
tfidf_text1 = vectorizer.transform(df['Solution'])
tfidf_text2 = vectorizer.transform(df['Answer MedVet'])
tfidf_text3 = vectorizer.transform(df['Answer ChatGPT'])

# Calculate the cosine similarity between each pair of vectors
# cosine_similarity returns a matrix, we take the diagonal where the pair similarities are found.
df['cosine_similarity_medvet'] = [cosine_similarity(tfidf_text1[i], tfidf_text2[i])[0][0] for i in range(len(df))]
df['cosine_similarity_chatgpt'] = [cosine_similarity(tfidf_text1[i], tfidf_text3[i])[0][0] for i in range(len(df))]

# Save the updated DataFrame back to CSV
df.to_csv('Results/Evaluation MedVet Cases combined scores.csv', index=False)  

df.head()

,Number,Case,Image,Solution,Source,Answer MedVet,Answer ChatGPT,cosine_similarity_medvet,cosine_similarity_chatgpt
0,1,Jake is a 3-year-old neutered German shepherd....,-,Periodontal disease is present to some degree ...,https://angelscampvetgroup.com/pet-dental-heal...,"Ah, I see. Based on the information provided,...","\nBased on the symptoms described for Jake, it...",0.274826,0.478195
1,2,Casey is a 14-year-old spayed domestic shortha...,-,If bacteria aren‚Äôt removed from under a pet‚...,https://angelscampvetgroup.com/pet-dental-heal...,Thank you for providing the details of Casey'...,"Based on the symptoms described for Casey, the...",0.336539,0.413385
2,3,Karl is a 5-year-old neutered greyhound. Karl‚...,-,As periodontal disease progresses to stage thr...,https://angelscampvetgroup.com/pet-dental-heal...,Thank you for providing me with the details of...,The symptoms Karl is experiencing suggest a de...,0.318073,0.305745
3,4,Dolly was a 14-year-old spayed female Chihuahu...,-,"When periodontal disease becomes advanced, sev...",https://angelscampvetgroup.com/pet-dental-heal...,Thank you for sharing the details of Dolly's ...,\nDolly's clinical presentation and findings f...,0.287016,0.365178
4,5,"A 13-year-old, female Shiba dog presented with...",Image Case 5,"In this case, MCT involved subcutaneous bleedi...",https://bvajournals.onlinelibrary.wiley.com/do...,Based on the provided case description and ac...,The information provided in the case of the 13...,0.103455,0.093578


<h2>Calculate BLEU Score</h2>

In [3]:
import pandas as pd
from sacrebleu.metrics import BLEU

# Load the CSV file
df = pd.read_csv('Results/Evaluation MedVet Cases combined scores.csv')  

# Function to calculate BLEU score
def calculate_bleu(ref, hyp):
    bleu = BLEU()  # Initialize the BLEU object
    score = bleu.corpus_score([hyp], [[ref]])  # Calculate the score
    return score.score

# Apply the function to each row in the DataFrame
df['BLEU_Score_medvet'] = df.apply(lambda row: calculate_bleu(row['Solution'], row['Answer MedVet']), axis=1)
df['BLEU_Score_chatgpt'] = df.apply(lambda row: calculate_bleu(row['Solution'], row['Answer ChatGPT']), axis=1)

# Save the DataFrame with the BLEU scores back to a CSV
df.to_csv('Results/Evaluation MedVet Cases combined scores.csv', index=False) 

df.head()

,Number,Case,Image,Solution,Source,Answer MedVet,Answer ChatGPT,cosine_similarity_medvet,cosine_similarity_chatgpt,BLEU_Score_medvet,BLEU_Score_chatgpt
0,1,Jake is a 3-year-old neutered German shepherd....,-,Periodontal disease is present to some degree ...,https://angelscampvetgroup.com/pet-dental-heal...,"Ah, I see. Based on the information provided,...","\nBased on the symptoms described for Jake, it...",0.274826,0.478195,1.549824,1.750375
1,2,Casey is a 14-year-old spayed domestic shortha...,-,If bacteria aren‚Äôt removed from under a pet‚...,https://angelscampvetgroup.com/pet-dental-heal...,Thank you for providing the details of Casey'...,"Based on the symptoms described for Casey, the...",0.336539,0.413385,0.684827,1.654700
2,3,Karl is a 5-year-old neutered greyhound. Karl‚...,-,As periodontal disease progresses to stage thr...,https://angelscampvetgroup.com/pet-dental-heal...,Thank you for providing me with the details of...,The symptoms Karl is experiencing suggest a de...,0.318073,0.305745,0.806335,1.161963
3,4,Dolly was a 14-year-old spayed female Chihuahu...,-,"When periodontal disease becomes advanced, sev...",https://angelscampvetgroup.com/pet-dental-heal...,Thank you for sharing the details of Dolly's ...,\nDolly's clinical presentation and findings f...,0.287016,0.365178,1.483485,0.813035
4,5,"A 13-year-old, female Shiba dog presented with...",Image Case 5,"In this case, MCT involved subcutaneous bleedi...",https://bvajournals.onlinelibrary.wiley.com/do...,Based on the provided case description and ac...,The information provided in the case of the 13...,0.103455,0.093578,0.836307,0.589361


<h2>Calculate Rouge Score</h2>

In [4]:
import pandas as pd
from rouge_score import rouge_scorer

# Load the CSV file
df = pd.read_csv('Results/Evaluation MedVet Cases combined scores.csv')  

# Function to compute ROUGE score
def compute_rouge(text1, text2):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(text1, text2)
    return scores['rougeL'].fmeasure

# Apply function and create a new column
df['rouge_score_medvet'] = df.apply(lambda row: compute_rouge(row['Solution'], row['Answer MedVet']), axis=1)
df['rouge_score_chatgpt'] = df.apply(lambda row: compute_rouge(row['Solution'], row['Answer ChatGPT']), axis=1)

# Save to CSV
df.to_csv('Results/Evaluation MedVet Cases combined scores.csv', index=False)

df.head()

,Number,Case,Image,Solution,Source,Answer MedVet,Answer ChatGPT,cosine_similarity_medvet,cosine_similarity_chatgpt,BLEU_Score_medvet,BLEU_Score_chatgpt,rouge_score_medvet,rouge_score_chatgpt
0,1,Jake is a 3-year-old neutered German shepherd....,-,Periodontal disease is present to some degree ...,https://angelscampvetgroup.com/pet-dental-heal...,"Ah, I see. Based on the information provided,...","\nBased on the symptoms described for Jake, it...",0.274826,0.478195,1.549824,1.750375,0.165829,0.175207
1,2,Casey is a 14-year-old spayed domestic shortha...,-,If bacteria aren‚Äôt removed from under a pet‚...,https://angelscampvetgroup.com/pet-dental-heal...,Thank you for providing the details of Casey'...,"Based on the symptoms described for Casey, the...",0.336539,0.413385,0.684827,1.654700,0.128405,0.128326
2,3,Karl is a 5-year-old neutered greyhound. Karl‚...,-,As periodontal disease progresses to stage thr...,https://angelscampvetgroup.com/pet-dental-heal...,Thank you for providing me with the details of...,The symptoms Karl is experiencing suggest a de...,0.318073,0.305745,0.806335,1.161963,0.106538,0.112033
3,4,Dolly was a 14-year-old spayed female Chihuahu...,-,"When periodontal disease becomes advanced, sev...",https://angelscampvetgroup.com/pet-dental-heal...,Thank you for sharing the details of Dolly's ...,\nDolly's clinical presentation and findings f...,0.287016,0.365178,1.483485,0.813035,0.138817,0.116071
4,5,"A 13-year-old, female Shiba dog presented with...",Image Case 5,"In this case, MCT involved subcutaneous bleedi...",https://bvajournals.onlinelibrary.wiley.com/do...,Based on the provided case description and ac...,The information provided in the case of the 13...,0.103455,0.093578,0.836307,0.589361,0.102041,0.077698


<h2>Calculate winner and from scores</h2>


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('Results/Evaluation MedVet Cases combined scores.csv')  

# Function to compute ROUGE score
def points medvet(score_medvet, score_gpt):
    return score_medvet > score_gpt

# Apply function and create a new column
df['rouge_score_medvet'] = df.apply(lambda row: is_medvet_winner(row['cosine_similarity_medvet'], row['Answer MedVet']), axis=1)